In [1]:
import sys
from pathlib import Path
import pandas as pd

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils
from config import date_str, DOWNLOAD_DIR, DEST_DIR

path_data = f'..\data\{date_str}_df_finviz_merged.parquet'
# path_corr = '..\data\df_corr_emv_matrix.parquet'
# path_cov = '..\data\df_cov_emv_matrix.parquet'
# path_output = f'..\picks\{date_str}_portf.txt'


print(utils.__file__)  # Should point to your src/utils.py
print(f'path_date: {path_data}')
# print(f'path_corr: {path_corr}')
# print(f'path_cov: {path_cov}')  
# print(f'path_output: {path_output}')
# print((f'date_str: {date_str}'))



Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


c:\Users\ping\Files_win10\python\py311\stocks\src\utils.py
path_date: ..\data\2025-04-16_df_finviz_merged.parquet


In [2]:
import pandas as pd
# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell
pd.set_option('display.max_rows', 100)
# pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.4f}'.format)

In [3]:
import pandas as pd

def get_column_values_above_threshold(df, column_name='Avg Volume, M', threshold=0.75):
  """
  Analyzes the number and percentage of values in a DataFrame column that are above a specified threshold,
  and returns the filtered DataFrame.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to analyze. Defaults to 'Avg Volume, M'.
    threshold (float): The threshold value to compare against. Defaults to 1.00.

  Returns:
    pd.DataFrame: A DataFrame containing only the rows where the specified column's value is above the threshold.
  """
  
  count_before = len(df)
  above_threshold_df = df[df[column_name] > threshold]
  count_after = len(above_threshold_df)
  percentage = (count_after / len(df)) * 100

  print(f"count_before: {count_before}")
  print(f"count_after above threshold ({threshold}): {count_after}")
  print(f"Percentage above threshold ({threshold}): {percentage:.2f}%")

  return above_threshold_df


In [4]:
df_data = pd.read_parquet(path_data)

# liquidity filter, Avg Volume, M > 0.75M
df_data = get_column_values_above_threshold(df_data, column_name='Avg Volume, M', threshold=0.75)

# Drop specified columns with NaNs in df_data
df_data = df_data.drop(['All-Time High %', 'All-Time Low %', 'Dividend %'], axis=1)

# df_corr = pd.read_parquet(path_corr)
# df_cov = pd.read_parquet(path_cov)

# print(f'\ndf_cov.shape: {df_cov.shape}')
# display(df_cov.head())

# print(f'\ndf_corr.shape: {df_corr.shape}')
# display(df_corr.head())

print(f'\ndf_data.shape: {df_data.shape}')
display(df_data.head())
display((df_data.describe()))

count_before: 1510
count_after above threshold (0.75): 1172
Percentage above threshold (0.75): 77.62%

df_data.shape: (1172, 52)


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",2924200.0000,1.2900,41.7300,-22.2700,-1.9581,-2.1100,-8.4800,-18.1700,-15.8400,12.7200,-6.0000,-12.5300,-14.7300,-22.1400,15.0400,-25.1600,18.6400,10.9500,5.6252,7.6200,4.7700,45.9526,62.8600,0.7400,194.6600,-3.7000,-12.4579,-11.7934,0.0000,2.4516,4.2994,1.5156,-0.7199,-1.3296,0.8845,-2.2811,-3.4643,0.6676,-2.1335,-3.0645,0.6697,-0.9873,-1.4313,0.8198,-0.8355,-1.1775,0.8407,0.5221,0.7708,1.1090
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2759490.0000,0.9700,43.3600,-11.9300,-4.3352,-4.9400,-3.2100,-12.9300,-11.4400,-10.2600,-2.5200,-5.2400,-11.1900,-11.4700,7.6600,-20.7400,7.6600,13.6600,3.6800,4.6800,3.2300,14.9240,25.4100,0.6000,371.2000,-3.7700,-15.1931,-12.7678,0.0000,-4.4585,-5.4663,0.4186,0.0513,0.1111,1.0102,-1.4499,-2.6887,0.7561,-1.3114,-2.2244,0.7692,-2.2158,-3.1944,0.6464,-0.9287,-1.3044,0.8358,-0.3565,-0.4914,0.9370
NVDA,NVIDIA Corp,"Technology, Semiconductors",2527230.0000,2.0800,43.2600,-22.8700,-5.8055,-9.4100,-10.2700,-23.9800,-24.9800,20.4300,-5.4800,-12.5900,-17.9800,-27.7900,19.5700,-32.3600,36.9900,7.8500,7.5794,8.2400,6.1400,342.4554,307.7700,1.1300,103.5700,-7.6900,-7.5915,-9.0574,0.1931,-2.4226,-3.0686,0.6249,1.0753,2.1852,1.2234,-0.6617,-1.1570,0.8798,-0.2660,-0.4399,0.9517,-1.4781,-1.9739,0.7652,-0.7240,-0.9852,0.8782,0.5887,0.8442,1.1057
GOOGL,Alphabet Inc,"Communication Services, Internet Content & Inf...",1881870.0000,1.0000,43.4800,-19.0000,-2.4246,-3.3900,-4.5700,-21.5900,-7.0500,-0.9900,-2.3500,-8.8000,-11.5100,-25.9500,9.1100,-25.9500,9.1100,6.1700,4.0240,4.9500,3.8100,19.7684,34.8100,0.5800,153.3300,-1.9100,-240.5883,-15.8573,0.0000,0.5996,1.0684,1.0953,0.9532,2.0217,1.1830,-1.5260,-2.4514,0.7646,-2.0080,-2.8829,0.7156,-2.5904,-3.3363,0.6438,-0.2673,-0.3821,0.9559,-0.0103,-0.0148,0.9982
GOOG,Alphabet Inc,"Communication Services, Internet Content & Inf...",1881090.0000,1.0000,43.7600,-18.3000,-2.4467,-3.4000,-4.3500,-21.0100,-6.4700,-0.4700,-2.2400,-8.5400,-11.0400,-25.4500,9.0600,-25.4500,9.0600,6.2000,3.9848,4.8500,3.7900,12.1475,22.3100,0.5500,155.5900,-1.9500,-152.9809,-15.8319,0.0000,0.1699,0.2886,1.0258,1.0115,2.1934,1.1940,-1.4307,-2.3379,0.7781,-1.9325,-2.8084,0.7255,-2.5442,-3.3144,0.6497,-0.2518,-0.3624,0.9585,0.0022,0.0032,1.0004


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000
mean,58048.0290,1.0000,45.1530,-5.9082,0.0517,-1.6719,-6.4288,-7.6225,-7.1064,8.6905,-3.2936,-6.7980,-6.6401,-16.9488,11.7125,-22.4054,27.3285,4.2838,4.4130,5.2802,3.9152,5.5117,6.6846,0.7687,97.5009,-1.1308,13.9788,143.9786,13.9336,5.2640,442.4125,56.6451,-1.0019,-1.0044,0.9167,-1.6568,-2.0640,0.7958,-0.9663,-1.1480,0.8820,-0.7199,-0.8385,0.9079,-0.3201,-0.3540,0.9558,0.3068,0.5173,1.0693
std,190139.7012,0.5684,7.9295,17.2101,2.4972,4.9198,8.6202,16.3435,20.4445,49.7864,4.9442,8.6720,14.5468,12.0202,7.6811,15.3516,53.9482,5.4832,2.3181,2.5139,1.9913,17.5794,15.5220,0.5641,110.2795,1.8116,657.1978,354.8401,31.5641,9.5300,1357.4378,171.0460,2.2690,3.5426,0.4045,2.0239,2.9689,0.3719,1.7352,2.6072,0.3474,1.6102,2.3025,0.2995,1.0197,1.4594,0.1870,0.8319,1.2763,0.1671
min,2980.0000,-0.9800,25.5400,-72.1400,-11.4045,-27.0900,-53.2900,-74.6200,-77.1900,-84.5900,-33.9800,-53.6200,-70.4700,-71.4800,-1.9000,-95.0800,-1.9000,0.0200,0.0298,0.0100,0.0100,0.2346,0.7505,0.2000,2.1900,-14.4200,-2913.5563,-15.8744,0.0000,-46.0103,-15.2100,0.0000,-10.0284,-9.2488,0.1692,-11.8170,-10.5040,0.1696,-6.3900,-7.0399,0.3643,-5.3925,-5.9490,0.3855,-3.4814,-3.9706,0.5156,-1.9607,-2.4750,0.6822
25%,8697.5000,0.6500,39.8600,-16.1150,-1.3519,-4.4725,-11.3100,-18.0700,-18.3750,-11.1550,-6.2125,-12.3000,-15.0775,-24.0075,7.4775,-32.0375,8.6050,1.1200,3.0293,3.8200,2.7900,0.9671,1.5100,0.5200,33.4275,-2.0225,-19.4442,-13.7479,0.0000,0.2257,0.3498,1.0384,-2.3168,-3.2586,0.6670,-2.9136,-3.8753,0.5926,-2.0062,-2.7404,0.6847,-1.9367,-2.5622,0.6899,-0.9877,-1.3296,0.8275,-0.2197,-0.3049,0.9615
50%,17525.0000,0.9700,43.5150,-6.6450,0.1201,-1.5650,-6.1600,-7.8300,-8.6350,2.7900,-3.0800,-6.6400,-6.7200,-15.1350,10.4600,-20.4300,13.7500,2.5500,4.0513,4.9500,3.6000,1.8458,2.7300,0.6700,62.7400,-1.0650,-7.4788,-8.9766,0.2003,4.2795,9.0762,2.0155,-0.9032,-1.4329,0.8493,-1.8733,-2.6781,0.7059,-1.2716,-1.7231,0.7885,-1.0293,-1.4230,0.8268,-0.4298,-0.5953,0.9214,0.2761,0.3909,1.0527
75%,46820.0000,1.2400,49.8475,2.4325,1.4596,1.3000,-0.8500,2.0650,0.7875,18.9000,-0.1800,-0.6825,0.6775,-7.0675,14.3175,-10.4175,29.7525,5.2125,5.5677,6.5825,4.9125,4.1990,5.7550,0.8800,113.2850,0.0000,3.8549,11.7423,2.0461,9.1843,34.9353,5.1802,0.2214,0.4226,1.0460,-0.6778,-0.9194,0.8891,-0.1162,-0.1554,0.9796,0.3496,0.5005,1.0633,0.3204,0.4649,1.0597,0.7751,1.1652,1.1536
max,2924200.0000,4.2800,88.3600,118.3700,10.1574,26.2300,44.0000,87.7300,125.4800,973.0800,23.9200,41.2900,70.8200,3.9000,85.0100,3.9000,1021.6100,47.9700,25.3491,25.3800,18.0600,342.4554,307.7700,13.8200,966.5900,6.1200,14785.9349,1007.3053,90.7379,88.3526,4952.3498,624.9374,8.4685,27.0874,3.8757,9.9963,22.8581,4.8998,6.1985,16.6375,3.4161,5.0091,10.6682,2.5231,3.1791,6.2988,1.8234,3.5634,7.1329,1.7634


In [5]:
import pandas as pd
import os
import datetime

# --- Configuration ---
STATS_HISTORY_FILE = 'market_daily_summary_stats_history.csv' # Where to store the history

def append_daily_stats(df_daily_data, date_str, filename=STATS_HISTORY_FILE):
    """
    Calculates descriptive stats for df_daily_data, flattens them,
    and appends/updates them as a row in a historical CSV file.

    Args:
        df_daily_data (pd.DataFrame): DataFrame containing the raw data for ALL stocks for ONE specific day.
        date_str (str): The date for this data in 'YYYY-MM-DD' format.
        filename (str): The path to the CSV file storing the historical summary stats.
    """
    print(f"Processing stats for date: {date_str}")

    # 1. Calculate descriptive statistics for the input day's data
    try:
        daily_stats_raw = df_daily_data.describe()
        print("Successfully calculated describe() stats.")
    except Exception as e:
        print(f"Error calculating describe() stats: {e}")
        return # Stop if calculation fails

    # 2. Flatten the describe() output into a single row DataFrame
    try:
        # Unstack turns the stats (mean, std, etc.) into columns alongside the original metrics
        stats_flat = daily_stats_raw.unstack().to_frame().T
        # Create clearer column names like 'Metric_Statistic' (e.g., 'RSI_mean', 'SMA50 %_50%')
        stats_flat.columns = ['_'.join(col).strip() for col in stats_flat.columns.values]
        # Add the date to this row
        stats_flat['Date'] = pd.to_datetime(date_str)
        # Set the Date as the index for this single row
        stats_flat = stats_flat.set_index('Date')
        print("Successfully flattened stats into a single row.")
        # print(stats_flat) # Optional: print the flattened stats for verification

    except Exception as e:
        print(f"Error flattening the stats DataFrame: {e}")
        return # Stop if flattening fails

    # 3. Load existing history, append/update, and save
    try:
        df_summary_history = pd.DataFrame() # Initialize empty
        if os.path.exists(filename):
            print(f"Loading existing history from: {filename}")
            df_summary_history = pd.read_csv(filename, index_col='Date', parse_dates=True)

        # Check if the date already exists - update if it does, append if it doesn't
        current_date = pd.to_datetime(date_str)
        if current_date in df_summary_history.index:
            print(f"Updating existing entry for {date_str}")
            # Update existing row(s) - combines columns, replacing old values for this date
            df_summary_history.update(stats_flat)
            # Alternative: df_summary_history.loc[current_date] = stats_flat.iloc[0] # Direct overwrite
        else:
            print(f"Appending new entry for {date_str}")
            df_summary_history = pd.concat([df_summary_history, stats_flat])

        # Ensure the history is sorted by date
        df_summary_history = df_summary_history.sort_index()

        # Save the updated history back to CSV
        df_summary_history.to_csv(filename, index=True) # index=True saves the Date index
        print(f"Successfully saved updated stats history to: {filename}")

    except Exception as e:
        print(f"Error loading/saving the stats history file '{filename}': {e}")



In [6]:
# Simulate processing for a couple of days
print("-" * 20, "Day 1", "-" * 20)
append_daily_stats(df_data, date_str)

-------------------- Day 1 --------------------
Processing stats for date: 2025-04-16
Successfully calculated describe() stats.
Successfully flattened stats into a single row.
Loading existing history from: market_daily_summary_stats_history.csv
Appending new entry for 2025-04-16
Successfully saved updated stats history to: market_daily_summary_stats_history.csv


In [7]:
print(f'date_str: {date_str}')
df_data.describe()

date_str: 2025-04-16


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000,1172.0000
mean,58048.0290,1.0000,45.1530,-5.9082,0.0517,-1.6719,-6.4288,-7.6225,-7.1064,8.6905,-3.2936,-6.7980,-6.6401,-16.9488,11.7125,-22.4054,27.3285,4.2838,4.4130,5.2802,3.9152,5.5117,6.6846,0.7687,97.5009,-1.1308,13.9788,143.9786,13.9336,5.2640,442.4125,56.6451,-1.0019,-1.0044,0.9167,-1.6568,-2.0640,0.7958,-0.9663,-1.1480,0.8820,-0.7199,-0.8385,0.9079,-0.3201,-0.3540,0.9558,0.3068,0.5173,1.0693
std,190139.7012,0.5684,7.9295,17.2101,2.4972,4.9198,8.6202,16.3435,20.4445,49.7864,4.9442,8.6720,14.5468,12.0202,7.6811,15.3516,53.9482,5.4832,2.3181,2.5139,1.9913,17.5794,15.5220,0.5641,110.2795,1.8116,657.1978,354.8401,31.5641,9.5300,1357.4378,171.0460,2.2690,3.5426,0.4045,2.0239,2.9689,0.3719,1.7352,2.6072,0.3474,1.6102,2.3025,0.2995,1.0197,1.4594,0.1870,0.8319,1.2763,0.1671
min,2980.0000,-0.9800,25.5400,-72.1400,-11.4045,-27.0900,-53.2900,-74.6200,-77.1900,-84.5900,-33.9800,-53.6200,-70.4700,-71.4800,-1.9000,-95.0800,-1.9000,0.0200,0.0298,0.0100,0.0100,0.2346,0.7505,0.2000,2.1900,-14.4200,-2913.5563,-15.8744,0.0000,-46.0103,-15.2100,0.0000,-10.0284,-9.2488,0.1692,-11.8170,-10.5040,0.1696,-6.3900,-7.0399,0.3643,-5.3925,-5.9490,0.3855,-3.4814,-3.9706,0.5156,-1.9607,-2.4750,0.6822
25%,8697.5000,0.6500,39.8600,-16.1150,-1.3519,-4.4725,-11.3100,-18.0700,-18.3750,-11.1550,-6.2125,-12.3000,-15.0775,-24.0075,7.4775,-32.0375,8.6050,1.1200,3.0293,3.8200,2.7900,0.9671,1.5100,0.5200,33.4275,-2.0225,-19.4442,-13.7479,0.0000,0.2257,0.3498,1.0384,-2.3168,-3.2586,0.6670,-2.9136,-3.8753,0.5926,-2.0062,-2.7404,0.6847,-1.9367,-2.5622,0.6899,-0.9877,-1.3296,0.8275,-0.2197,-0.3049,0.9615
50%,17525.0000,0.9700,43.5150,-6.6450,0.1201,-1.5650,-6.1600,-7.8300,-8.6350,2.7900,-3.0800,-6.6400,-6.7200,-15.1350,10.4600,-20.4300,13.7500,2.5500,4.0513,4.9500,3.6000,1.8458,2.7300,0.6700,62.7400,-1.0650,-7.4788,-8.9766,0.2003,4.2795,9.0762,2.0155,-0.9032,-1.4329,0.8493,-1.8733,-2.6781,0.7059,-1.2716,-1.7231,0.7885,-1.0293,-1.4230,0.8268,-0.4298,-0.5953,0.9214,0.2761,0.3909,1.0527
75%,46820.0000,1.2400,49.8475,2.4325,1.4596,1.3000,-0.8500,2.0650,0.7875,18.9000,-0.1800,-0.6825,0.6775,-7.0675,14.3175,-10.4175,29.7525,5.2125,5.5677,6.5825,4.9125,4.1990,5.7550,0.8800,113.2850,0.0000,3.8549,11.7423,2.0461,9.1843,34.9353,5.1802,0.2214,0.4226,1.0460,-0.6778,-0.9194,0.8891,-0.1162,-0.1554,0.9796,0.3496,0.5005,1.0633,0.3204,0.4649,1.0597,0.7751,1.1652,1.1536
max,2924200.0000,4.2800,88.3600,118.3700,10.1574,26.2300,44.0000,87.7300,125.4800,973.0800,23.9200,41.2900,70.8200,3.9000,85.0100,3.9000,1021.6100,47.9700,25.3491,25.3800,18.0600,342.4554,307.7700,13.8200,966.5900,6.1200,14785.9349,1007.3053,90.7379,88.3526,4952.3498,624.9374,8.4685,27.0874,3.8757,9.9963,22.8581,4.8998,6.1985,16.6375,3.4161,5.0091,10.6682,2.5231,3.1791,6.2988,1.8234,3.5634,7.1329,1.7634


In [8]:
print(f'date_str: {date_str}')
df_data.head()

date_str: 2025-04-16


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",2924200.0000,1.2900,41.7300,-22.2700,-1.9581,-2.1100,-8.4800,-18.1700,-15.8400,12.7200,-6.0000,-12.5300,-14.7300,-22.1400,15.0400,-25.1600,18.6400,10.9500,5.6252,7.6200,4.7700,45.9526,62.8600,0.7400,194.6600,-3.7000,-12.4579,-11.7934,0.0000,2.4516,4.2994,1.5156,-0.7199,-1.3296,0.8845,-2.2811,-3.4643,0.6676,-2.1335,-3.0645,0.6697,-0.9873,-1.4313,0.8198,-0.8355,-1.1775,0.8407,0.5221,0.7708,1.1090
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2759490.0000,0.9700,43.3600,-11.9300,-4.3352,-4.9400,-3.2100,-12.9300,-11.4400,-10.2600,-2.5200,-5.2400,-11.1900,-11.4700,7.6600,-20.7400,7.6600,13.6600,3.6800,4.6800,3.2300,14.9240,25.4100,0.6000,371.2000,-3.7700,-15.1931,-12.7678,0.0000,-4.4585,-5.4663,0.4186,0.0513,0.1111,1.0102,-1.4499,-2.6887,0.7561,-1.3114,-2.2244,0.7692,-2.2158,-3.1944,0.6464,-0.9287,-1.3044,0.8358,-0.3565,-0.4914,0.9370
NVDA,NVIDIA Corp,"Technology, Semiconductors",2527230.0000,2.0800,43.2600,-22.8700,-5.8055,-9.4100,-10.2700,-23.9800,-24.9800,20.4300,-5.4800,-12.5900,-17.9800,-27.7900,19.5700,-32.3600,36.9900,7.8500,7.5794,8.2400,6.1400,342.4554,307.7700,1.1300,103.5700,-7.6900,-7.5915,-9.0574,0.1931,-2.4226,-3.0686,0.6249,1.0753,2.1852,1.2234,-0.6617,-1.1570,0.8798,-0.2660,-0.4399,0.9517,-1.4781,-1.9739,0.7652,-0.7240,-0.9852,0.8782,0.5887,0.8442,1.1057
GOOGL,Alphabet Inc,"Communication Services, Internet Content & Inf...",1881870.0000,1.0000,43.4800,-19.0000,-2.4246,-3.3900,-4.5700,-21.5900,-7.0500,-0.9900,-2.3500,-8.8000,-11.5100,-25.9500,9.1100,-25.9500,9.1100,6.1700,4.0240,4.9500,3.8100,19.7684,34.8100,0.5800,153.3300,-1.9100,-240.5883,-15.8573,0.0000,0.5996,1.0684,1.0953,0.9532,2.0217,1.1830,-1.5260,-2.4514,0.7646,-2.0080,-2.8829,0.7156,-2.5904,-3.3363,0.6438,-0.2673,-0.3821,0.9559,-0.0103,-0.0148,0.9982
GOOG,Alphabet Inc,"Communication Services, Internet Content & Inf...",1881090.0000,1.0000,43.7600,-18.3000,-2.4467,-3.4000,-4.3500,-21.0100,-6.4700,-0.4700,-2.2400,-8.5400,-11.0400,-25.4500,9.0600,-25.4500,9.0600,6.2000,3.9848,4.8500,3.7900,12.1475,22.3100,0.5500,155.5900,-1.9500,-152.9809,-15.8319,0.0000,0.1699,0.2886,1.0258,1.0115,2.1934,1.1940,-1.4307,-2.3379,0.7781,-1.9325,-2.8084,0.7255,-2.5442,-3.3144,0.6497,-0.2518,-0.3624,0.9585,0.0022,0.0032,1.0004
